In [3]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import json 
import fiona
from osgeo import ogr, gdal


def create_polygon(coords):
    ring = ogr.Geometry(ogr.wkbLinearRing)
    for coord in coords:
        for xy in coord:
            ring.AddPoint(xy[0],xy[1])
            poly = ogr.Geometry(ogr.wkbPolygon)
            poly.AddGeometry(ring)
    return poly.ExportToIsoWkt()

def create_shp_with_geojson(json,geo_type):
    count = 0 
    gdal.SetConfigOption("GDAL_FILENAME_IS_UTF8", "YES")
    gdal.SetConfigOption("SHAPE_ENCODING", "UTF-8")
    driver = ogr.GetDriverByName("ESRI Shapefile")
    properties_temp=list(json['features'][0]['properties'])
    
    if geo_type=='Polygon' or geo_type=='MultiPolygon':
        #修改的地方
        schema = {
            'geometry':'Polygon',
        }
        Shp = fiona.open('C:/Users/timchen/Desktop/notu class/專題/geojson_to_shapefile/cropPolygon.shp', mode='w', driver='ESRI Shapefile',
        schema = schema, crs = "EPSG:4326")
    elif geo_type=='Point': #建立shapefile的檔案名稱,屬性表格格式
        #修改的地方
        schema = {
            'geometry':'Point',
            'properties':[('Name','str')]
        }
        Shp = fiona.open('C:/Users/timchen/Desktop/notu class/專題/geojson_to_shapefile/cropPoints.shp', mode='w', driver='ESRI Shapefile',
        schema = schema, crs = "EPSG:4326") 
    elif geo_type == 'LineString':
        #修改的地方
        schema = {
            'geometry':'LineString',
            'properties':[('Name','str')]
        }        
        Shp = fiona.open('C:/Users/timchen/Desktop/notu class/專題/geojson_to_shapefilecropLineString.shp', mode='w', driver='ESRI Shapefile',
        schema = schema, crs = "EPSG:4326")
        
        
    for record in json['features']:
        #print(record)
        geo = record.get("geometry")
        geo_type = geo.get('type')
        temp=list(record['properties'])
        if geo_type == 'Polygon':
            #修改的地方
            xyList = []
            for i in range(len(record['geometry']['coordinates'][0])):
                xyList.append(record['geometry']['coordinates'][0][i])
            rowDict = {
                'geometry' : {'type':'Polygon',
                'coordinates': [xyList]}, 
            }
            try : 
                Shp.write(rowDict)
                count+=1
            except :
                 print("does not write in ")
                    
        elif geo_type == 'MultiPolygon':
            xyList = []
            for y in range(len(record['geometry']['coordinates'])):          
                for i in range(len(record['geometry']['coordinates'][y])):
                        xyList.append(record['geometry']['coordinates'][y][i])
                        rowDict = {
                                    'geometry' : {'type':'Polygon',
                                    'coordinates': [xyList]}, 
                        }
                try : 
                    Shp.write(rowDict)
                    count+=1
                except :
                     print("does not write in ")
        elif geo_type == 'Point':
            #修改的地方
 
            ex = record['geometry']['coordinates']
            rowDict = {
                'geometry' : {'type':'Point',
                'coordinates': ex },
                'properties': {'Name' : record['properties']['NAME_C']},
            }
            try : 
                Shp.write(rowDict)
                count+=1
            except :
                 print("does not write in ")
            #print(rowDict)
            
        elif geo_type == 'LineString':
            #修改的地方
            xyList = []
            for i in range(len(record['geometry']['coordinates'][0])):
                xyList.append(record['geometry']['coordinates'][0][i])
            rowDict = {
                'geometry' : {'type':'Polygon',
                'coordinates': [xyList]}, 
            }
            try : 
                Shp.write(rowDict)
                count+=1
            except :
                 print("does not write in ")
        else:
            print('Could not discern geometry')

    print("total data input : " ,count)
    Shp.close()

In [4]:
with open(r"C:/Users/timchen/Desktop/notu class/專題/geojson_to_shapefile/地下水觀測井位置圖_彰化縣現存站.json", encoding="UTF-8") as jsonfile: #讀取json檔案
    data = json.load(jsonfile)
    geo_type=str(data['features'][0]['geometry']['type'])
    temp=list(data['features'][0]['properties'])
    #print(temp)
    create_shp_with_geojson(data,geo_type)

total data input :  90


In [5]:
import geopandas as gpd
gdf_Rail=gpd.read_file(r'C:/Users/timchen/Desktop/notu class/專題/geojson_to_shapefile/cropPoints.shp',decoding='utf-8', newline='')            
    #print(shpReader)
print(gdf_Rail)

     Name                    geometry
0   國聖(1)  POINT (120.56913 24.09266)
1   國聖(2)  POINT (120.56913 24.09266)
2   國聖(3)  POINT (120.56913 24.09266)
3   東芳(1)  POINT (120.51599 24.06277)
4   東芳(2)  POINT (120.51599 24.06277)
..    ...                         ...
85  下柑(2)  POINT (120.55074 23.82143)
86     過溪  POINT (120.57213 23.81033)
87     榮光  POINT (120.60506 23.81508)
88  溪州(1)  POINT (120.50126 23.85506)
89  溪州(2)  POINT (120.50126 23.85506)

[90 rows x 2 columns]


C:\Users\timchen\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:35: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  out = from_shapely(data)


In [9]:
with open(r"C:/Users/timchen/Desktop/notu class/專題/geojson_to_shapefile/choushui_twd97.json", encoding="UTF-8") as jsonfile: #讀取json檔案
    data = json.load(jsonfile)
    geo_type=str(data['features'][0]['geometry']['type'])
    temp=list(data['features'][0]['properties'])
    #print(temp)
    create_shp_with_geojson(data,geo_type)

total data input :  1


In [11]:
import geopandas as gpd
gdf_Rail=gpd.read_file(r"C:/Users/timchen/Desktop/notu class/專題/geojson_to_shapefile/cropPolygon.shp",decoding='utf-8', newline='')            
    #print(shpReader)
print(gdf_Rail)

   FID                                           geometry
0    0  POLYGON ((120.44078 24.15664, 120.46561 24.159...
